In [741]:
# def dice_avg(dice_size) -> float:
#     return (dice_size + 1) / 2.0
# 
# dice_avg(4), dice_avg(6), dice_avg(8), dice_avg(10), dice_avg(12)

In [742]:
import math
import random

ATTACK_ROLL_DICE_SIZE = 20
BASE_ABILITY_SIZE = 10
DEFAULT_TARGET_AC = 13


def roll_dice(dice_size=ATTACK_ROLL_DICE_SIZE) -> int:
    return random.randint(1, dice_size)


[roll_dice() for _ in range(10)]


[10, 14, 15, 14, 19, 10, 4, 1, 18, 18]

In [743]:
# def base_weapon_damage_avg(dice_size, dice_count=1, bonus=0):
#     return (dice_avg(dice_size) + bonus) * dice_count
# 
# base_weapon_damage_avg(10, bonus=1), base_weapon_damage_avg(6, bonus=1)

In [744]:
from dataclasses import dataclass


@dataclass
class Weapon:
    name: str
    dice_size: int
    dice_count: int = 1
    bonus: int = 0

    def damage_roll(self, critical=False, proficiency_bonus=0) -> int:
        dice_count = self.dice_count * 2 if critical else self.dice_count
        return sum([roll_dice(self.dice_size) for _ in range(dice_count)]) + self.bonus + proficiency_bonus


hand_crossbow_0 = Weapon("Hand Crossbow", dice_size=6)
hand_crossbow_1 = Weapon("Hand Crossbow +1", dice_size=6, bonus=1)
hand_crossbow_2 = Weapon("Hand Crossbow +2", dice_size=6, bonus=2)
heavy_crossbow_0 = Weapon("Heavy Crossbow", dice_size=10)
heavy_crossbow_1 = Weapon("Heavy Crossbow +1", dice_size=10, bonus=1)
heavy_crossbow_2 = Weapon("Heavy Crossbow +2", dice_size=10, bonus=2)
hand_crossbow_0.damage_roll(), hand_crossbow_0.damage_roll(), hand_crossbow_0.damage_roll(), hand_crossbow_0.damage_roll(
    critical=True)

(1, 4, 4, 11)

In [745]:
longbow_0 = Weapon("Longbow", dice_size=8)
longbow_1 = Weapon("Longbow", dice_size=8, bonus=1)
longbow_2 = Weapon("Longbow", dice_size=8, bonus=2)
longbow_3 = Weapon("Longbow", dice_size=8, bonus=3)
longbow_3.damage_roll(), longbow_3.damage_roll(critical=True)

(10, 13)

In [746]:
heavy_crossbow_2.damage_roll(critical=True), heavy_crossbow_2.damage_roll(critical=True), heavy_crossbow_2.damage_roll(
    critical=True),

(12, 12, 14)

In [747]:
proficiency_bonus_on_level = {
    1: 2,
    2: 2,
    3: 2,
    4: 2,
    5: 3,
    6: 3,
    7: 3,
    8: 3,
    9: 4,
    10: 4,
    11: 4,
    12: 4
}

In [748]:
main_ability_on_level_default = {
    1: 17,
    2: 17,
    3: 18,
    4: 18,
    5: 18,
    6: 19,
    7: 19,
    8: 19,
    9: 20,
    10: 20,
    11: 20,
    12: 21
}

In [749]:
import sys
import logging
from pathlib import Path

SHOULD_LOG = False

logger = logging.Logger(">>")
stream_handler = logging.StreamHandler(sys.stdout)
stream_handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s %(name)s [%(levelname)s]: %(message)s')
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

path_to_log = Path(
    r"C:\Users\slafniy\Desktop\bg3modding\DedTunedProject\Data\Projects\CommonUtility\damage_calculator.log")
file_handler = logging.FileHandler(path_to_log, mode='w')
logger.addHandler(file_handler)


def do_nothing(*args, **kwargs):
    pass


if not SHOULD_LOG:
    logger.info = do_nothing

In [750]:
class Character:
    def __init__(self, name: str,
                 level: int,
                 weapon_main: Weapon,
                 weapon_offhand: Weapon = None,
                 fighting_style_archery=False,
                 fighting_style_dual_weapon=False):
        self.name = name
        self.level = level
        self.weapon_main = weapon_main
        self.weapon_offhand = weapon_offhand
        self.base_proficiency_bonus = proficiency_bonus_on_level[level]
        self.ability_proficiency_bonus = int(math.floor((main_ability_on_level_default[level] - BASE_ABILITY_SIZE) / 2))
        self.has_fighting_style_archery = fighting_style_archery
        self.has_fighting_style_dual_weapon = fighting_style_dual_weapon

    def do_attack(self, weapon: Weapon, target_ac, apply_proficiency_bonus=True) -> int:
        attack_roll = roll_dice(ATTACK_ROLL_DICE_SIZE)
        if attack_roll == 1:
            logger.info(f"{self.name} -> 0 damage, Critical miss!")
            return 0

        if attack_roll == ATTACK_ROLL_DICE_SIZE:
            res = weapon.damage_roll(critical=True,
                                     proficiency_bonus=self.ability_proficiency_bonus if apply_proficiency_bonus else 0)
            logger.info(f"{self.name} -> {res} damage, Critical hit!")
            return res

        style_bonus = 2 if self.has_fighting_style_archery else 0
        attack_roll = attack_roll + self.base_proficiency_bonus + self.ability_proficiency_bonus + weapon.bonus + style_bonus
        if attack_roll < target_ac:
            logger.info(f"{self.name} -> 0 damage, rolled {attack_roll} against {target_ac}")
            return 0

        res = weapon.damage_roll(proficiency_bonus=self.ability_proficiency_bonus if apply_proficiency_bonus else 0)
        logger.info(f"{self.name} -> {res} damage, rolled {attack_roll} against {target_ac}")
        return res

    def main_hand_attack(self, target_ac=DEFAULT_TARGET_AC):
        return self.do_attack(self.weapon_main, target_ac)

    def offhand_attack(self, target_ac=DEFAULT_TARGET_AC):
        return self.do_attack(self.weapon_offhand, target_ac, self.has_fighting_style_dual_weapon)
    
    def simulate(self, rounds=5, target_ac=DEFAULT_TARGET_AC, iterations=1000):
        results = []
        for _ in range(iterations):
            results_per_round = []
            for _ in range(rounds):
                dpr = self.main_hand_attack(target_ac)
                dpr += self.offhand_attack(target_ac) if self.weapon_offhand is not None else 0
                dpr += self.main_hand_attack(target_ac) if self.level >= 5 else 0
                results_per_round.append(dpr)
            results.append(sum(results_per_round) / rounds)
        avg_damage = sum(results) / iterations
        print(f'{self.name} lvl {self.level} with {self.weapon_main.name} '
              f'{"and " if self.weapon_offhand is not None else ""}'
              f'{self.weapon_offhand.name if self.weapon_offhand is not None else ""}, avg DPR: {avg_damage:.1f}')
    


In [752]:
Character("Ranger archery", 5, heavy_crossbow_1, fighting_style_archery=True).simulate(rounds=15)
Character("Ranger duals", 5, hand_crossbow_1, hand_crossbow_1, fighting_style_dual_weapon=True).simulate(rounds=15)



Ranger archery lvl 5 with Heavy Crossbow +1 , avg DPR: 19.4
Ranger duals lvl 5 with Hand Crossbow +1 and Hand Crossbow +1, avg DPR: 20.9
